# Projeto 2 - Ciência dos Dados

Nome: Isadora Stigliani Dalberto

Nome: Matheus Amaral Ricardo

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [1]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [2]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle
from IPython.display import display

___
## Autenticando no  Twitter

* Conta: ***@MatheuseDora1***

In [3]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @MatheuseDora1

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [4]:
#Produto escolhido:
#produto = 'Era uma vez em Hollywood'

#Quantidade mínima de mensagens capturadas:
#n = 500
#Quantidade mínima de mensagens para a base de treinamento:
#t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
#lang = 'pt'

Capturando os dados do twitter:

In [5]:
#Cria um objeto para a captura
#api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
#i = 1
#msgs = []
#for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
#    msgs.append(msg.text.lower())
#    i += 1
#    if i > n:
#        break

#Embaralhando as mensagens para reduzir um possível viés
#shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [6]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
#if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
#    #Abre o arquivo para escrita
#    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
#    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
#    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

#    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
#    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
#    writer.save()

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

Abre o arquivo com a coluna "Relevância" adicionada manualmente no Excel

In [7]:
treinamento = pd.read_excel('Era uma vez em Hollywood.xlsx', sheet_name="Treinamento")
teste = pd.read_excel('Era uma vez em Hollywood.xlsx', sheet_name="Teste")

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento.

In [8]:
import re 

def cleanup(text):
    import string
    punctuation = '[!-.:?;\s@]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed 

print() #Espaçamento

**Separação dos tweets entre Relevante (1) e Irrelevante(0)**

In [9]:
treinamento_relevancia1 = treinamento.loc[treinamento['Relevancia']==1,:]
treinamento_relevancia0 = treinamento.loc[treinamento['Relevancia']==0,:]
print() #Espaçamento

**Tranforma todos os caracteres em Minúsculas**

In [10]:
treinamento_string1 = treinamento_relevancia1.Treinamento.str.lower()
treinamento_string0 = treinamento_relevancia0.Treinamento.str.lower()
print() #Espaçamento

**Limpa o Texto (retira caracteres especiais)**

In [11]:
treinamento_clean1=treinamento_string1.apply(cleanup)
treinamento_clean0=treinamento_string0.apply(cleanup)
print() #Espaçamento

**Tranforma a tabela em um Textão**

In [12]:
treinamento_textao1 = treinamento_clean1.str.cat(sep=' ', na_rep=' ')
treinamento_textao0 = treinamento_clean0.str.cat(sep=' ', na_rep=' ')
print() #Espaçamento
total = treinamento_textao1 + treinamento_textao0

**Separa as palavras do texto**

In [13]:
treinamento_split1 = pd.Series(treinamento_textao1.split())
treinamento_split0 = pd.Series(treinamento_textao0.split())
serie_total = pd.Series(total.split())
print() #Espaçamento

**Value Counts para os Tweets Relevantes**

In [14]:
treinamento1_contagem = treinamento_split1.value_counts()
treinamento1_contagem_relativa = treinamento_split1.value_counts(True)
print() #Espaçamento

**Value Counts para os Tweets não Relevantes**

In [40]:
treinamento0_contagem = treinamento_split0.value_counts()
treinamento0_contagem_relativa = treinamento_split0.value_counts(True)
print() #Espaçamento

**P(0)**

In [16]:
tabela_total_absuluta = serie_total.value_counts(False)
p0 = treinamento0_contagem.sum()/tabela_total_absuluta.sum()
p0

0.33813353282214326

**P(1)**

In [17]:
tabela_total_absuluta = serie_total.value_counts(False)
p1 = treinamento1_contagem.sum()/tabela_total_absuluta.sum()
p1

0.6618664671778567

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

$$
P(0|frase) = \frac{P(frase|0)P(0)}{P(frase)}
$$


$$
P(1|frase) = \frac{P(frase|1)P(1)}{P(frase)}
$$

**Transforma todos os caracteres do Teste em Minúscula**

In [18]:
teste_string = teste.Teste.str.lower()

**Limpa o Texto (retira caracteres especiais)**

In [19]:
teste_clean = teste_string.apply(cleanup)

**Tranforma a tabela em um Textão**

In [20]:
teste_textao = teste_clean.str.cat(sep=' ', na_rep=' ')

**Separa as palavras do texto**

In [21]:
teste_split = pd.Series(teste_textao.split())

**Value Counts para os Tweets Relevantes**

In [22]:
teste_contagem = teste_split.value_counts()
teste_contagem_relativa = teste_split.value_counts(True)

**Código de separação**

In [37]:
irrelv = []
irrelf = []
relv = []
relf = []

for frase in teste.loc[teste['Relevancia']==1,:]['Teste']:
    pf0 = ((treinamento0_contagem[frase.split()]+1)/len(serie_total)).prod()
    p0f = (pf0 * p0) / teste_contagem_relativa[frase.split()].prod()
    pf1 = ((treinamento1_contagem[frase.split()]+1)/len(serie_total)).prod()
    p1f = (pf1 * p1) / teste_contagem_relativa[frase.split()].prod()
    if p1f>p0f:
        relv.append(frase)
    elif p1f<p0f:
        relf.append(frase)

In [38]:
for frase in teste.loc[teste['Relevancia']==0,:]['Teste']:
    pf0 = ((treinamento0_contagem[frase.split()]+1)/len(serie_total)).prod()
    p0f = (pf0 * p0) / teste_contagem_relativa[frase.split()].prod()
    pf1 = ((treinamento1_contagem[frase.split()]+1)/len(serie_total)).prod()
    p1f = (pf1 * p1) / teste_contagem_relativa[frase.split()].prod()
    if p1f>p0f:
        irrelf.append(frase)
    elif p1f<p0f:
        irrelv.append(frase)

In [46]:
irrelvporc = len(irrelv)/len(teste)*100
print('Verdadeiros Negativos: {0}%'.format(irrelvporc))
irrelfporc = len(irrelf)/len(teste)*100
print('Falsos Negativos: {0}%'.format(irrelfporc))
relfporc = len(relf)/len(teste)*100
print('Falsos Positivos: {0}%'.format(relfporc))
relvporc = len(relv)/len(teste)*100
print('Verdadeiros Positivos: {0}%'.format(relvporc))

Verdadeiros Negativos: 15.5%
Falsos Negativos: 37.0%
Falsos Positivos: 18.5%
Verdadeiros Positivos: 28.999999999999996%


## Conclusão:

   Neste trabalho realizamos a análise de tweets relacionados ao filme "Era Uma Vez em Hollywood", e, através de tweets classificados entre relevantes ou não relevantes, criamos um classificador de novos tweets, que a partir da análise probabilística das palavras contidas naqueles, os classificaria em relevantes ou não relevantes.
    
   Após a análise das porcentagens de tweets verdadeiros positivos (mensagens relevantes marcadas como relevantes), verdadeiros negativos (mensagens irrelevantes marcadas como irrelevantes), falsos positivos (mensagens irrelevantes marcadas como relevantes) e falsos negativos (mensagens relevantes marcadas como irrelevantes), podemos concluir que nosso classificador nâo foi eficaz, visto que as porcentagens de falsos, somadas, equivalem a 55,5% das mensagens.
    
   Os verdadeiros positivos são quase o dobro dos verdadeiros negativos e os falsos negativos são a maioria devido às mensagens com dupla negação ou sarcasmo, que acabam sendo classificadas como negativas, já que nosso trabalho basea-se em probabilidade do aparecimento de palavras, logo são tratadas apenas como números, não pelo sentido da frase, o que acaba classificando mensagens que seriam positivas como negativas. Devido aos mesmos fatores que o número de verdadeiros positivos é maior do que falsos positivos, mas o número de falsos negativos é maior do que de verdadeiros negativos.
    
   Contudo, o financiamento de nosso projeto deve ser alongado pela empresa de modo que possamos chegar a um classificador mais eficiente (que classifique tweets em tempo real e ampliar o número de categorias), visto que sabemos que para melhorá-lo devemos levar em consideração sarcasmo, dupla negação, caracteres acoplados, logo sabemos como alcançá-lo, para que a empresa possa se beneficiar ao obter informações sobre a opinião das pessoas sobre o filme, o tipo de pessoa que gosta deste, como melhorar a partir dos comentários negativos e não perder tempo com mensagens irrelevantes, que atrapalham este processo.

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**